Il k-nearest neighbors è un algoritmo di classificazione basato sull'assunto che le osservazioni simili appartengano alla stessa classe. Consiste nel calcolare la distanza tra un'osservazione di test e tutte le osservazioni del dataset di training e nell'assegnare l'etichetta della classe più presente tra i k osservazioni più vicine.

Il processo è questo:

1) Scegliere il numero di vicini k da considerare durante il calcolo delle distanze

2) Calcolare la distanza tra l'osservazione di test e tutte le osservazioni del dataset di training

3) Selezionare le k osservazioni più vicine

4) Assegnare l'etichetta della classe più presente tra le k osservazioni più vicine all'osservazione di test

Questo algoritmo può essere lento nel caso di dataset di grandi dimensioni e l'accuratezza del modello può essere influenzata in modo significativo dalla scelta del numero di vicini k.


Pro: 

Semplicità: è un algoritmo semplice e facile da capire e implementare

Buona prestazione con dataset di piccole dimensioni

Contro:

Lentezza con dataset di grandi dimensioni perchè deve calcolare la distanza tra l'osservazione di test e tutte le osservazioni ogni volta. Infatti è considerato un algoritmo lazy learner, ossia un algoritmo che richiede meno tempo per l'apprendimento e più tempo per la classificazione

Sensibilità alla scelta del numero k di vicini

Sensibilità alla presenza di outliers

In [21]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)

set.seed(1)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_finale.Rdata")
df <- df_finale

In [22]:
names(df) <- c('Centre', 'Gender', 'Age', 'LHOSpretx', 'Status', 'VaricealBleed', 'Ascites', 'Tips',  
                'PVthrombosis', 'NutritStatus', 'Previoussurg', 'Ventilationpretx', 'Vasopressors', 
               'InfusionAlb', 'PaO2FiO2', 'PlatCount', 'Albumin', 'Duration', 'PRBCintraop', 'FFPintraop', 'Plateletsintraop',
               'Fibrinogen', 'CRIO', 'Crystalloids', 'PPS5', 'Norepinend', 'Lactates', 'ColdIschemia', 'ReperfusSy', 'PCshunt',
               'Cavaveinanasth', 'VVBypass', 'Biliaryanasth', 'Openabdomen', 'Donorgender', 'Donortype', 'Organtype' ,'DonorECMO', 
               'RegionalPerf', 'OrganBiopsy', 'Machineperf', 'DonorSite', 'DonorCauseDeath', 'DonorAge', 'DonorHeight', 'DonorSodiemia',
               'DonorGGT', 'MELDNa', 'Alcoholic', 'cirrhosisDrugs', 'NASH', 'Lateretx', 'DonorMaastricht', 'DonorSteatosis' ,
               'HypoNormoter', 'Nocomorbodities', 'Hypertension', 'SeverePsychosis', 'Periodstx', 'Nighttx', 'Admsite' ,
               'Encephalopathy', 'Cholestatic', 'Rare', 'Viral', 'HCC', 'Respcomorbidities', 'CVcomorbidities', 'Neurocomorbidities',
               'Renalcomorbidities', 'Diabetes', 'Anticoagulation', 'MELDExcept', 'deg_cat')

# Feature selection

Feature selection fatta attraverso l'algoritmo boruta che si basa sull'algoritmo di random forest e utilizza una tecnica di confronto dei risultati ottenuti con un insieme di dati di controllo per determinare quali variabili sono significative. è un algoritmo che tiene conto delle interazioni tra variabili, è versatile in quanto lavora sia su variabili continue che categoriche ed è resistente agli outliers

In [23]:
library(Boruta)
# Decide if a variable is important or not using Boruta
boruta_output <- Boruta(deg_cat ~ ., data=df, doTrace=2)  # perform Boruta search


boruta_signif <- names(boruta_output$finalDecision[boruta_output$finalDecision %in% c("Confirmed", "Tentative")])  # collect Confirmed and Tentative variables
print(boruta_signif)  # significant variables


plot(boruta_output, cex.axis=1.2, las=2, xlab="", main="Variable Importance")  # plot variable importance


df_boruta <- df[,boruta_signif]

df_boruta <- apply(select_if(df,is.matrix),as.numeric)


 1. run of importance source...

 2. run of importance source...

 3. run of importance source...

 4. run of importance source...

 5. run of importance source...

 6. run of importance source...

 7. run of importance source...

 8. run of importance source...

 9. run of importance source...

 10. run of importance source...

 11. run of importance source...

 12. run of importance source...

 13. run of importance source...

After 13 iterations, +1.1 mins: 

 confirmed 12 attributes: Albumin, Centre, Duration, Lactates, LHOSpretx and 7 more;

 rejected 39 attributes: Age, Alcoholic, Anticoagulation, Biliaryanasth, Cavaveinanasth and 34 more;

 still have 22 attributes left.


 14. run of importance source...

 15. run of importance source...

 16. run of importance source...

 17. run of importance source...

 18. run of importance source...

After 18 iterations, +1.3 mins: 

 rejected 4 attributes: Machineperf, NASH, Status, Viral;

 still have 18 attributes left.


 19. run of im

 [1] "Centre"           "LHOSpretx"        "Ascites"          "NutritStatus"    
 [5] "InfusionAlb"      "PaO2FiO2"         "PlatCount"        "Albumin"         
 [9] "Duration"         "PRBCintraop"      "FFPintraop"       "Plateletsintraop"
[13] "CRIO"             "Crystalloids"     "PPS5"             "Norepinend"      
[17] "Lactates"         "ReperfusSy"       "Openabdomen"      "DonorSodiemia"   
[21] "MELDNa"           "Nocomorbodities"  "Admsite"          "Encephalopathy"  
[25] "HCC"             


ERROR: Error in match.fun(FUN): l'argomento "FUN"  non è specificato e non ha un valore predefinito


In [24]:
par(mar = c(15, 5, 2, 0.1))  
plot(boruta_output, cex.axis=1.1, las=2, xlab="", main="Variable Importance")

In [25]:
df_boruta$deg_cat <- df$deg_cat

In [26]:
boruta_signif #variabili selezionate

[1] "Centre"           "LHOSpretx"        "Ascites"          "NutritStatus"    
 [5] "InfusionAlb"      "PaO2FiO2"         "PlatCount"        "Albumin"         
 [9] "Duration"         "PRBCintraop"      "FFPintraop"       "Plateletsintraop"
[13] "CRIO"             "Crystalloids"     "PPS5"             "Norepinend"      
[17] "Lactates"         "ReperfusSy"       "Openabdomen"      "DonorSodiemia"   
[21] "MELDNa"           "Nocomorbodities"  "Admsite"          "Encephalopathy"  
[25] "HCC"

## Data partition

Viene effettuata divisione tra training e test set, che è del 70-30 per tutti i modelli utilizzati

In [27]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df$deg_cat, SplitRatio = 0.7)
train  <- subset(df, sample == TRUE)
test   <- subset(df, sample == FALSE)



TestClasses <- test$deg_cat
TestData <- subset(test,select= -deg_cat)



Viene effettuato un bilanciamento della variabile target perchè in origine la classe ottimo conteneva circa 1/4 dei record rispetto alle altre due classi e i modelli facevano fatica a classificare quel determinato valore. La funzione utilizzata è SmoteClassif() che:

1) Seleziona a caso una osservazione della classe minoritaria e ne calcola la distanza rispetto alle altre osservazioni della stessa classe

2) Seleziona a caso un'altra osservazione della classe minoritaria e ne calcola la distanza rispetto alle altre osservazioni della stessa classe

3) Genera una nuova osservazione sintetica che si trova a metà strada tra le due osservazioni selezionate in precedenza

4) Ripete i passaggi precedenti fino a raggiungere il bilanciamento desiderato


Questa funzione utilizza il knn per calcolare le distanze tra le osservazioni e generare le osservazioni, infatti il valore di k è stato selezionato pari alla radice quadrata del numero totale di record presenti nel training set come si fa per il knn normalmente per il knn



In [28]:
train <- SmoteClassif(deg_cat ~ ., train, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(train$deg_cat)

TrainClasses <- train$deg_cat
TrainData <- subset(train,select= -deg_cat)


   fast NotFast 
    441     442 

Viene effettuato lo stesso processo considerando il dataset dopo la boruta feature selection invece del dataset con tutte le feature in modo da poter fare una comparazione sui risultati successivamente

In [29]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df_boruta$deg_cat, SplitRatio = 0.7)
trainB  <- subset(df_boruta, sample == TRUE)
testB   <- subset(df_boruta, sample == FALSE)



TestClassesB <- testB$deg_cat
TestDataB <- subset(testB,select= -deg_cat)


In [30]:
trainB <- SmoteClassif(deg_cat ~ ., trainB, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(trainB$deg_cat)

TrainClassesB <- trainB$deg_cat
TrainDataB <- subset(trainB,select= -deg_cat)


   fast NotFast 
    441     442 

## Models

Attraverso la funzione trainControl viene effettuata una repeated cross validation e una random search.

La prima permette di ottenere una stima più accurata dell'accuratezza del modello:

1) Dividere il dataset in un numero predeterminato di folds (in questo caso è stato scelto n=10 quindi 10 folds)

2) Eseguire la cross-validation ripetutamente utilizzando ogni fold come test set e gli altri folds come training set

3) Calcolare la media e la deviazione standard delle misure di accuratezza ottenute per ogni fold

4) Valutare l'accuratezza e la robustezza del modello utilizzando la media e la deviazione standard delle misure di accuratezza

L'utilizzo della repeated cross validation è particolarmente importante quando si lavora con dataset di dimensioni ridotte o con un numero limitato di osservazioni perchè in questi casi i risultati possono essere influenzata dall'aleatorietà nella divisione dei dati in training set e test set. La repeated cross-validation permette quindi di ottenere una stima più accurata dell'accuratezza del modello ripetendo il processo  più volte e considerando dati diversi



La random search, invece è una tecnica di ottimizzazione dei modelli che viene utilizzata per selezionare i migliori parametri di un modello. Consiste nel generare a caso diverse combinazioni di parametri e nel valutare l'accuratezza del modello per ogni combinazione. La combinazione di parametri che ottiene il risultato migliore viene quindi selezionata come quella ottimale. Questa soluzione è stata preferita alla grid search che poteva dare risultati migliori per una questione di costo computazionale

In [31]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

Viene allenato il modello knn sul dataset senza feature selection

In [32]:
model_knn <- train(x=TrainData, y=TrainClasses, 
               method = "kknn",
               trControl = ctrl) 

+ Fold01.Rep1: kmax=145, distance=0.2166, kernel=epanechnikov 
- Fold01.Rep1: kmax=145, distance=0.2166, kernel=epanechnikov 
+ Fold01.Rep1: kmax= 91, distance=0.7301, kernel=cos 
- Fold01.Rep1: kmax= 91, distance=0.7301, kernel=cos 
+ Fold01.Rep1: kmax=171, distance=1.5548, kernel=cos 
- Fold01.Rep1: kmax=171, distance=1.5548, kernel=cos 
+ Fold02.Rep1: kmax=145, distance=0.2166, kernel=epanechnikov 
- Fold02.Rep1: kmax=145, distance=0.2166, kernel=epanechnikov 
+ Fold02.Rep1: kmax= 91, distance=0.7301, kernel=cos 
- Fold02.Rep1: kmax= 91, distance=0.7301, kernel=cos 
+ Fold02.Rep1: kmax=171, distance=1.5548, kernel=cos 
- Fold02.Rep1: kmax=171, distance=1.5548, kernel=cos 
+ Fold03.Rep1: kmax=145, distance=0.2166, kernel=epanechnikov 
- Fold03.Rep1: kmax=145, distance=0.2166, kernel=epanechnikov 
+ Fold03.Rep1: kmax= 91, distance=0.7301, kernel=cos 
- Fold03.Rep1: kmax= 91, distance=0.7301, kernel=cos 
+ Fold03.Rep1: kmax=171, distance=1.5548, kernel=cos 
- Fold03.Rep1: kmax=171, dis

Viene allenato il modello knn sul dataset con feature selection

In [33]:
model_knnB <- train(x=TrainDataB, y=TrainClassesB, 
               method = "kknn",
               trControl = ctrl) 

+ Fold01.Rep1: kmax=279, distance=2.015, kernel=triweight 
- Fold01.Rep1: kmax=279, distance=2.015, kernel=triweight 
+ Fold01.Rep1: kmax=167, distance=2.168, kernel=inv 
- Fold01.Rep1: kmax=167, distance=2.168, kernel=inv 
+ Fold01.Rep1: kmax= 51, distance=1.297, kernel=inv 
- Fold01.Rep1: kmax= 51, distance=1.297, kernel=inv 
+ Fold02.Rep1: kmax=279, distance=2.015, kernel=triweight 
- Fold02.Rep1: kmax=279, distance=2.015, kernel=triweight 
+ Fold02.Rep1: kmax=167, distance=2.168, kernel=inv 
- Fold02.Rep1: kmax=167, distance=2.168, kernel=inv 
+ Fold02.Rep1: kmax= 51, distance=1.297, kernel=inv 
- Fold02.Rep1: kmax= 51, distance=1.297, kernel=inv 
+ Fold03.Rep1: kmax=279, distance=2.015, kernel=triweight 
- Fold03.Rep1: kmax=279, distance=2.015, kernel=triweight 
+ Fold03.Rep1: kmax=167, distance=2.168, kernel=inv 
- Fold03.Rep1: kmax=167, distance=2.168, kernel=inv 
+ Fold03.Rep1: kmax= 51, distance=1.297, kernel=inv 
- Fold03.Rep1: kmax= 51, distance=1.297, kernel=inv 
+ Fold04.R

I parametri testati sono: 

kmax = numero di nearest neighbors considerati

distance = parametro per la distanza minkowski

kernel = per specificare il metodo di con cui vengono calcolati i pesi da utilizzare durante il calcolo delle distanze tra le osservazioni

## Results

Vengono analizzati i risultati ottenuti dai due modelli durante la cross validation. Le metriche utilizzate sono: accuracy e kappa. La prima si riferisce alla percentuale di osservazioni correttamente classificate dal modello rispetto al totale di osservazioni. La seconda viene utilizzata per misurare l'accordo tra più modelli di classificazione. Può assumere valori compresi tra -1 e 1. Un valore di kappa vicino a 1 indica un alto livello di accordo tra le classificazioni, mentre un valore di kappa vicino a 0 indica un accordo simile a quello ottenuto per caso. Un valore di kappa negativo indica un accordo peggiore di quello ottenuto per caso.

In [34]:
print(model_knn)
plot(model_knn)

k-Nearest Neighbors 

883 samples
 73 predictor
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 795, 795, 794, 793, 795, 795, ... 
Resampling results across tuning parameters:

  kmax  distance   kernel        Accuracy   Kappa    
   91   0.7301072  cos           0.6887289  0.3776954
  145   0.2165729  epanechnikov  0.7000170  0.4002882
  171   1.5547635  cos           0.6865191  0.3732520

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were kmax = 145, distance = 0.2165729
 and kernel = epanechnikov.


In [35]:
print(model_knnB)
plot(model_knnB)

k-Nearest Neighbors 

883 samples
 25 predictor
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 794, 794, 795, 794, 795, 795, ... 
Resampling results across tuning parameters:

  kmax  distance  kernel     Accuracy   Kappa    
   51   1.296544  inv        0.7260853  0.4521438
  167   2.167873  inv        0.7317926  0.4633479
  279   2.015433  triweight  0.7464888  0.4929495

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were kmax = 279, distance = 2.015433
 and kernel = triweight.


Vengono messi a confronto i risultati ottenuti sulla cross validation dei due modelli in modo da selezionare il più performante da utilizzare nel test set

In [36]:
# collect resamples
results <- resamples(list(knn=model_knn,knn_boruta=model_knnB))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: knn, knn_boruta 
Number of resamples: 10 

Accuracy 
                Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
knn        0.5909091 0.6554307 0.7102273 0.7000170 0.7386364 0.8068182    0
knn_boruta 0.6629213 0.7106742 0.7443182 0.7464888 0.7897727 0.8181818    0

Kappa 
                Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
knn        0.1818182 0.3127673 0.4204545 0.4002882 0.4772727 0.6136364    0
knn_boruta 0.3271169 0.4194397 0.4886364 0.4929495 0.5795455 0.6363636    0


## Prediction

Viene selezionato il modello più performante per usarlo nel test set ed avere risultati definitivi.

In [37]:
pred_knn <- predict(model_knnB,test)  
pred_knn.prob <- predict(model_knnB,test, type="prob")


result <- confusionMatrix(TestClasses,  pred_knn) 
result
result$byClass

Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast      82      45
   NotFast   89     163
                                         
               Accuracy : 0.6464         
                 95% CI : (0.596, 0.6946)
    No Information Rate : 0.5488         
    P-Value [Acc > NIR] : 7.181e-05      
                                         
                  Kappa : 0.2694         
                                         
 Mcnemar's Test P-Value : 0.0002035      
                                         
            Sensitivity : 0.4795         
            Specificity : 0.7837         
         Pos Pred Value : 0.6457         
         Neg Pred Value : 0.6468         
             Prevalence : 0.4512         
         Detection Rate : 0.2164         
   Detection Prevalence : 0.3351         
      Balanced Accuracy : 0.6316         
                                         
       'Positive' Class : fast           
                                   

Sensitivity          Specificity       Pos Pred Value 
           0.4795322            0.7836538            0.6456693 
      Neg Pred Value            Precision               Recall 
           0.6468254            0.6456693            0.4795322 
                  F1           Prevalence       Detection Rate 
           0.5503356            0.4511873            0.2163588 
Detection Prevalence    Balanced Accuracy 
           0.3350923            0.6315930

In [38]:
roc_knn <- multiclass.roc(TestClasses, as.numeric(pred_knn))
roc_knn



Setting direction: controls < cases




Call:
multiclass.roc.default(response = TestClasses, predictor = as.numeric(pred_knn))

Data: as.numeric(pred_knn) with 2 levels of TestClasses: fast, NotFast.
Multi-class area under the curve: 0.6462

In [39]:
roc_obj <- roc(TestClasses, as.numeric(pred_knn))
plot(roc_obj, main="Curva ROC")

Setting levels: control = fast, case = NotFast

Setting direction: controls < cases



In [40]:
roc_obj


Call:
roc.default(response = TestClasses, predictor = as.numeric(pred_knn))

Data: as.numeric(pred_knn) in 127 controls (TestClasses fast) < 252 cases (TestClasses NotFast).
Area under the curve: 0.6462

Probabilità per ogni classe

In [41]:
pred_knn.prob

fast,NotFast
0.97939281,0.020607185
0.51747146,0.482528538
0.02730498,0.972695019
0.63639516,0.363604838
0.22031840,0.779681596
0.34344438,0.656555624
0.68887746,0.311122537
0.22421959,0.775780409
0.58227301,0.417726985
0.81101758,0.188982420
